In [23]:
import pandas as pd
import numpy as np

dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':float, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}

kc_data = pd.read_csv('c://users/intel/Desktop/Coursera/Regression/week5/kc_house_data.csv', dtype=dtype_dict)
training = pd.read_csv('c://users/intel/Desktop/Coursera/Regression/week5/wk3_kc_house_train_data.csv', dtype=dtype_dict)
validation = pd.read_csv('c://users/intel/Desktop/Coursera/Regression/week5/wk3_kc_house_valid_data.csv', dtype=dtype_dict)
testing = pd.read_csv('c://users/intel/Desktop/Coursera/Regression/week5/wk3_kc_house_test_data.csv', dtype=dtype_dict)

In [2]:
from math import log, sqrt
kc_data['sqft_living_sqrt'] = kc_data['sqft_living'].apply(sqrt)
kc_data['sqft_lot_sqrt'] = kc_data['sqft_lot'].apply(sqrt)
kc_data['bedrooms_square'] = kc_data['bedrooms']*kc_data['bedrooms']
kc_data['floors_square'] = kc_data['floors']*kc_data['floors']

In [3]:
from sklearn import linear_model  # using scikit-learn

all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

model_all = linear_model.Lasso(alpha=5e2, normalize=True) # set parameters
model_all.fit(kc_data[all_features], kc_data['price']) # learn weights

Lasso(alpha=500.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=True, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [13]:
model_all.coef_

array([     0.        ,      0.        ,      0.        ,    134.43931396,
            0.        ,      0.        ,      0.        ,      0.        ,
            0.        ,      0.        ,  24750.00458561,      0.        ,
        61749.10309071,      0.        ,      0.        ,     -0.        ,
            0.        ])

In [19]:
for i in range(0, len(all_features)): 
    print (all_features[i], '=',  model_all.coef_[i])

bedrooms = 0.0
bedrooms_square = 0.0
bathrooms = 0.0
sqft_living = 134.439313955
sqft_living_sqrt = 0.0
sqft_lot = 0.0
sqft_lot_sqrt = 0.0
floors = 0.0
floors_square = 0.0
waterfront = 0.0
view = 24750.0045856
condition = 0.0
grade = 61749.1030907
sqft_above = 0.0
sqft_basement = 0.0
yr_built = -0.0
yr_renovated = 0.0


In [28]:
testing['sqft_living_sqrt'] = testing['sqft_living'].apply(sqrt)
testing['sqft_lot_sqrt'] = testing['sqft_lot'].apply(sqrt)
testing['bedrooms_square'] = testing['bedrooms']*testing['bedrooms']
testing['floors_square'] = testing['floors']*testing['floors']

training['sqft_living_sqrt'] = training['sqft_living'].apply(sqrt)
training['sqft_lot_sqrt'] = training['sqft_lot'].apply(sqrt)
training['bedrooms_square'] = training['bedrooms']*training['bedrooms']
training['floors_square'] = training['floors']*training['floors']

validation['sqft_living_sqrt'] = validation['sqft_living'].apply(sqrt)
validation['sqft_lot_sqrt'] = validation['sqft_lot'].apply(sqrt)
validation['bedrooms_square'] = validation['bedrooms']*validation['bedrooms']
validation['floors_square'] = validation['floors']*validation['floors']

In [42]:
RES_LIST = []
for l1_penalty in np.logspace(1, 7, num=13):
    model = linear_model.Lasso(alpha=l1_penalty, normalize=True)
    model.fit(training[all_features], training['price']) # learn weights
    residuals = model.predict(validation[all_features]) - validation['price']
    RSS = sum(residuals**2)
    RES_LIST.append([l1_penalty, RSS])
    print (l1_penalty, '\t', RSS)

10.0 	 3.982133273e+14
31.6227766017 	 3.99041900253e+14
100.0 	 4.29791604073e+14
316.227766017 	 4.63739831045e+14
1000.0 	 6.45898733634e+14
3162.27766017 	 1.22250685943e+15
10000.0 	 1.22250685943e+15
31622.7766017 	 1.22250685943e+15
100000.0 	 1.22250685943e+15
316227.766017 	 1.22250685943e+15
1000000.0 	 1.22250685943e+15
3162277.66017 	 1.22250685943e+15
10000000.0 	 1.22250685943e+15


In [59]:
RES_LIST[:][:]

[[10.0, 398213327300134.19],
 [31.622776601683793, 399041900253348.5],
 [100.0, 429791604072557.94],
 [316.22776601683796, 463739831045119.5],
 [1000.0, 645898733633810.13],
 [3162.2776601683795, 1222506859427156.8],
 [10000.0, 1222506859427156.8],
 [31622.776601683792, 1222506859427156.8],
 [100000.0, 1222506859427156.8],
 [316227.76601683791, 1222506859427156.8],
 [1000000.0, 1222506859427156.8],
 [3162277.6601683795, 1222506859427156.8],
 [10000000.0, 1222506859427156.8]]

In [62]:
model = linear_model.Lasso(alpha=10.0, normalize=True)
model.fit(training[all_features], training['price']) # learn weights
residuals_test = model.predict(testing[all_features]) - testing['price']
RSS = sum(residuals_test**2)
print (RSS)

9.84674025527e+13


In [63]:
print(np.count_nonzero(model.coef_) + np.count_nonzero(model.intercept_) )

15


# limit ourselves to 7 features

In [65]:
max_nonzeros = 7

In [74]:
coef_list = []
l1_penalty_min = 0
l1_penalty_max = 100000
for l1_penalty in np.logspace(1, 4, num=20):
    model = linear_model.Lasso(alpha=l1_penalty, normalize=True)
    model.fit(training[all_features], training['price']) # learn weights
    non_zeros = np.count_nonzero(model.coef_) + np.count_nonzero(model.intercept_)
    coef_list.append([l1_penalty, non_zeros])
    if non_zeros > max_nonzeros:
        if l1_penalty_min <= l1_penalty:
            l1_penalty_min=l1_penalty
    if non_zeros < max_nonzeros:
        if l1_penalty_max >= l1_penalty:
            l1_penalty_max=l1_penalty   
print (l1_penalty_min, l1_penalty_max)
print (coef_list[:][:])

127.42749857 263.665089873
[[10.0, 15], [14.384498882876629, 15], [20.691380811147901, 15], [29.763514416313178, 15], [42.813323987193932, 13], [61.584821106602639, 12], [88.586679041008225, 11], [127.42749857031335, 10], [183.29807108324357, 7], [263.66508987303581, 6], [379.26901907322497, 6], [545.55947811685144, 6], [784.75997035146065, 5], [1128.8378916846884, 3], [1623.776739188721, 3], [2335.7214690901214, 2], [3359.8182862837812, 1], [4832.9302385717519, 1], [6951.9279617756056, 1], [10000.0, 1]]


In [84]:
coef_list_det = []
for l1_penalty in np.linspace(l1_penalty_min, l1_penalty_max, 20):
    model = linear_model.Lasso(alpha=l1_penalty, normalize=True)
    model.fit(training[all_features], training['price']) # learn weights
    non_zeros = np.count_nonzero(model.coef_) + np.count_nonzero(model.intercept_)
    if non_zeros==max_nonzeros:
        residuals_valid = model.predict(validation[all_features]) - validation['price']
        RSS = sum(residuals_valid**2)
        coef_list_det.append([l1_penalty, non_zeros, RSS])

In [82]:
coef_list_det[:]

[[156.10909673930755, 7, 440037365263317.13],
 [163.27949628155611, 7, 440777489641605.38],
 [170.44989582380464, 7, 441566698090139.0],
 [177.6202953660532, 7, 442406413188665.31],
 [184.79069490830176, 7, 443296716874313.06],
 [191.96109445055032, 7, 444239780526141.31],
 [199.13149399279888, 7, 445230739842613.94]]

In [85]:
final_model = linear_model.Lasso(alpha=156.10909673930755, normalize=True)
final_model.fit(training[all_features], training['price'])
for i in range(0, len(all_features)): 
    print (all_features[i], '=',  final_model.coef_[i])

bedrooms = -0.0
bedrooms_square = -0.0
bathrooms = 10610.8902844
sqft_living = 163.380251648
sqft_living_sqrt = 0.0
sqft_lot = -0.0
sqft_lot_sqrt = -0.0
floors = 0.0
floors_square = 0.0
waterfront = 506451.687115
view = 41960.0435549
condition = 0.0
grade = 116253.5537
sqft_above = 0.0
sqft_basement = 0.0
yr_built = -2612.23488036
yr_renovated = 0.0
